In [1]:
import trtpy as tp
import os
import cv2

In [2]:
engine_file = "yolov5m_2.fp32.trtmodel"
if not os.path.exists(engine_file):
    tp.compile_onnx_to_file(5, tp.onnx_hub("yolov5m"), engine_file)

yolo   = tp.Yolo(engine_file, tp.YoloType.V5, 0, 0.25)

In [3]:
yolo

In [4]:
image  = cv2.imread("inference/zand.jpg")
bboxes = yolo.commit(image).get()
bboxes

[<ObjectBox: left=438.02, top=434.06, right=521.42, bottom=719.22, class_label=27, confidence=0.81457>,
 <ObjectBox: left=750.17, top=40.92, right=1141.72, bottom=709.11, class_label=0, confidence=0.90385>,
 <ObjectBox: left=136.21, top=200.16, right=1110.42, bottom=713.44, class_label=0, confidence=0.72617>]

In [5]:
yolo

In [6]:
yolo.valid

True

In [7]:
import torch
import torchvision.models as models

In [8]:
input = torch.full((5, 3, 224, 224), 0.5)
m = models.resnet18(True).eval()
trt_model = tp.convert_torch_to_trt(m, input)

In [9]:
tout = m(input)
rtout = trt_model(input)
(tout - rtout).abs().max()

tensor(7.6294e-06, grad_fn=<MaxBackward1>)

In [10]:
import time

In [14]:
ntest = 100
t0 = time.time()
with torch.no_grad():
    for i in range(ntest):
        tout = m(input)
fee = time.time() - t0
print(f"{fee*1000:.2f} ms")

4527.91 ms


In [15]:
ntest = 100
t0 = time.time()
with torch.no_grad():
    for i in range(ntest):
        tout = trt_model(input)
fee = time.time() - t0
print(f"{fee*1000:.2f} ms")

440.81 ms


In [13]:
tp.convert_torch_to_trt?

Signature: tp.convert_torch_to_trt(torch_model, input, max_batch_size=None) -> trtpy.libtrtpyc.Infer
Docstring: <no docstring>
File:      /datav/newbb/lean/anaconda3/envs/torch1.8/lib/python3.9/site-packages/trtpy-1.0-py3.9.egg/trtpy/__init__.py
Type:      function
